# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [2]:
model.load_state_dict(torch.load('../../../Models/best_model3.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

In [6]:
camera = Camera.instance(width=224, height=224)

In [7]:
image = widgets.Image(format='jpeg', width=224, height=224)
pao_slider = widgets.FloatSlider(description='pao', min=0.0, max=1.0, orientation='vertical')
move_slider = widgets.FloatSlider(description='move', min=0.0, max=1.0, orientation='vertical')
hand_slider = widgets.FloatSlider(description='hand', min=0.0, max=1.0, orientation='vertical')
environment_slider = widgets.FloatSlider(description='environment', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, pao_slider,move_slider, hand_slider, environment_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [8]:
from jetbot import Robot

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [9]:
import time
import ipywidgets.widgets as widgets
from jetbot import Robot
import traitlets
import multiprocessing
import sys 
import threading

In [10]:
import torch.nn.functional as F
import playsound
import numpy as np
import threading
import torch
from jetbot import Robot
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

In [16]:
def loose_sequence():
    pW = multiprocessing.Process(target=playsound.playsound, args=("../Music/game_over.mp3",))
    pW.start()
    robot.left(0.1)
    while pW.is_alive():
        i = 1
    robot.stop()

In [17]:
def win_sequence():
    pW = multiprocessing.Process(target=playsound.playsound, args=("../Music/win.mp3",))
    pW.start()
    robot.left(0.1)
    while pW.is_alive():
        i = 1
    robot.stop()

In [18]:
def detect_hand(pG):
    x = preprocess(camera.value)
    y = model(x)
    y = F.softmax(y, dim=1)
    
    prob_hand = float(y.flatten()[1])
    hand_slider.value = prob_hand
    
    if prob_hand > 0.9:
        pG.terminate()
        win_sequence()
        #time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
        return 1
    else:
        return 0

In [19]:
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

In [20]:
#import imutils
def detect_movement():
    firstFrame = camera.value
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(gray, (33, 33), 0)
    start_time = time.time()
    frame_count = 0
    while True:
        frame_count = frame_count + 1
        frame = camera.value
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        text = "Static"
        if frame is None:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (33, 33), 0)
        frameDelta = cv2.absdiff(firstFrame, gray)
        thresh = cv2.threshold(frameDelta, 50, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = grab_contours(cnts)
        for c in cnts:
            if cv2.contourArea(c) < 85:
                continue
            text = "Movement"
            if text == "Movement":
                print(frame_count)
                return "LOSER"
        if text == "Movement":
            print(frame_count)
            return "LOSER"
        if time.time() >= start_time + 5:
            print(frame_count)
            return "CONTINUE"
        time.sleep(0.2)

In [25]:
def take_thresh(t,f):
    s_t = time.time()
    s_img = camera.value
    rgb_weights = [0.2989, 0.5870, 0.1140]
    s_img = np.dot(s_img, rgb_weights)
    
    diff_a = []

    while time.time() - s_t < t:
        t_img = np.dot(camera.value, rgb_weights)
        diff = np.sum(np.abs(t_img - s_img))/(224*224)
        diff_a.append(diff)
    
    diff = np.sum(diff_a)/len(diff_a)
    diff = f*diff
    return diff

In [26]:
def estel_move(threshold, N):
    s_t = time.time()
    s_img = camera.value
    rgb_weights = [0.2989, 0.5870, 0.1140]
    s_img = np.dot(s_img, rgb_weights)
    rows = np.shape(s_img)[0]
    cols = np.shape(s_img)[1]
    move = False
    p_m = []

    while time.time() - s_t < 5 and move == False:
        c_f = 0
        c_t = int(cols/N)
        t_img = np.dot(camera.value, rgb_weights)
        for i in range(N):
            s_img_i = s_img[:,c_f:c_t]
            t_img_i = t_img[:,c_f:c_t]
            #print(np.shape(s_img_i))
            diff = np.sum(np.abs(t_img_i - s_img_i))/(224*224)
            #print(diff)
            if diff > threshold:
                move = True
                p_m.append(i + 1)
                #i = N
            c_f = c_t + 1
            c_t = int(2*c_t)
        
    if move == False:
        print("no move")
    else:
        for i in range(len(p_m)):
            print("move player " + str(p_m[i]))
    
    return move, p_m
    

In [27]:
#main loop

In [28]:
def curator_role(T, N):
    total_players = N
    print("AAA")
    play = True
    while play:
        robot.stop()
        x = preprocess(camera.value)
        y = model(x)
        y = F.softmax(y, dim=1)

        prob_pao = float(y.flatten()[3])
        prob_move = float(y.flatten()[2])
        prob_hand = float(y.flatten()[1])
        prob_environment = float(y.flatten()[0])

        pao_slider.value = prob_pao
        move_slider.value = prob_move
        hand_slider.value = prob_hand
        environment_slider.value = prob_environment

        if prob_pao > 0.94:# and prob_move < 0.1:
            pG = multiprocessing.Process(target=playsound.playsound, args=("../Music/greenlight.mp3",))
            pG.start()
            while pG.is_alive():
                win = detect_hand(pG)
            if win == 1:
                print("end game")
                play = False#break
            else:
                robot.right(0.15)
                time.sleep(1.19)
                robot.stop()
                time.sleep(1.0)
                
                Move, p_m = estel_move(T,N) #Detect movement
                
                if Move == True and N == 1:#detect_movement() == "LOSER":
                    loose_sequence()
                    play = False
                elif Move == True and N > 1:
                    for i in range(len(p_m)):
                        num = p_m[i]
                        playsound.playsound("../Music/P"+str(num)+"_E.mp3")
                        total_players = total_players - 1
                    if total_players <= 0:
                        loose_sequence()
                        play = False
                    else:
                        robot.right(0.15)
                        time.sleep(1.14)
                        robot.stop()  
                else:
                    robot.right(0.15)
                    time.sleep(1.14)
                    robot.stop()
                    
                #T = T * 1.1 # increase threshold as players move closer
                    
        elif prob_hand > 0.99:# and prob_move < 0.05:
            win_sequence()
            break #sys.exit()
        else:
            robot.right(0.12)

In [29]:
#Controller

In [30]:
robot = Robot()

In [31]:
controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

In [32]:
left_link1 = traitlets.dlink((controller.axes[0], 'value'), (robot.left_motor, 'value'), transform=lambda x: x/8)
left_link2 = traitlets.dlink((controller.axes[0], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x/8)
left_link3 = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x/8)
left_link3 = traitlets.dlink((controller.axes[1], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x/8)

In [33]:
def callbackFunc(change,):
    global A, B, C, D, E, P1, P2, P3, P4, robot, T, N
    if change['owner'].buttons[8].pressed == True and A == True:
        A = False
        playsound.playsound("../Music/NotMove.mp3")
        T = take_thresh(3,1.1)
        print(T)
        playsound.playsound("../Music/PressSTART.mp3")
    if change['owner'].buttons[9].pressed == True and B == True:
        B = False
        robot.right(0.15)
        time.sleep(1.1)
        robot.stop()
        curator_role(T, N)
    if change['owner'].buttons[2].pressed == True and C == True:
        C = False
        playsound.playsound("../Music/greenlight.mp3")
    if change['owner'].buttons[1].pressed == True and D == True:
        playsound.playsound("../Music/game_over.mp3")
        D = False
    if change['owner'].buttons[3].pressed == True and E == True:
        E = False
        playsound.playsound("../Music/win.mp3")
    if change['owner'].buttons[4].pressed == True and P1 == True:
        P1 = False
        playsound.playsound("../Music/P1_E.mp3")
    if change['owner'].buttons[5].pressed == True and P2 == True:
        playsound.playsound("../Music/P2_E.mp3")
        P2 = False
    if change['owner'].buttons[6].pressed == True and P3 == True:
        P3 = False
        playsound.playsound("../Music/P3_E.mp3")
    if change['owner'].buttons[7].pressed == True and P4 == True:
        P4 = False
        playsound.playsound("../Music/P4_E.mp3")
    if change['owner'].buttons[10].pressed == True:# and N2 == True:
        #N2 = False
        N = 2
    if change['owner'].buttons[11].pressed == True:# and N3 == True:
        #N3 = False
        N = 3
    if change['owner'].buttons[0].pressed == True:
        N = 1
        A = True
        B = True
        C = True
        D = True
        E = True
        P1 = True
        P2 = True
        P3 = True
        P4 = True


In [34]:
#global robot
robot = Robot()

A = True
B = True
C = True
D = True
E = True
P1 = True
P2 = True
P3 = True
P4 = True

N = 1 #number of players
controller.observe(callbackFunc)

In [8]:
robot = Robot()
robot.stop()